**Resource**

https://github.com/hooshvare/parsgpt

In [1]:
!pip install -qU transformers
!pip install -qU hazm

In [2]:
import hazm

normalizer = hazm.Normalizer(persian_numbers=False)

def normalize_input(text):
    text = normalizer.normalize(text)
    return text

In [3]:
import json
import os
import glob

import numpy as np
import pandas as pd

from tqdm import tqdm
import pandas as pd
import numpy as np

from pathlib import Path

import torch
from transformers import AutoTokenizer, AutoConfig, AutoModelWithLMHead
from transformers import AutoTokenizer, GPT2LMHeadModel, GPT2Config

from IPython import display

In [4]:
path = "HooshvareLab/gpt2-fa"

tokenizer = AutoTokenizer.from_pretrained(
    path,
    bos_token='<s>',
    eos_token='</s>',
    pad_token='<pad>',
    unk_token='<unk>'
)
tokenizer.add_special_tokens({
    "bos_token": '</s>',
    "eos_token": '</s>',
    "pad_token": '<pad>',
    "unk_token": '<unk>'
})

config = AutoConfig.from_pretrained(
    path,
    bos_token_id=tokenizer("<s>")["input_ids"][0],
    eos_token_id=tokenizer("</s>")["input_ids"][0],
    pad_token_id=tokenizer("<pad>")["input_ids"][0],
    unk_token_id=tokenizer("<unk>")["input_ids"][0],
)

tokenizer.save_pretrained("/content/gpt2/")
config.save_pretrained("/content/gpt2/")

!wget "https://huggingface.co/HooshvareLab/gpt2-fa/resolve/main/pytorch_model.bin" -P /content/gpt2/
!wget "https://huggingface.co/HooshvareLab/gpt2-fa/resolve/main/tokenizer.json" -P /content/gpt2/

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:88: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


tokenizer_config.json:   0%|          | 0.00/728 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/808 [00:00<?, ?B/s]

vocab.json:   0%|          | 0.00/1.16M [00:00<?, ?B/s]

merges.txt:   0%|          | 0.00/875k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/2.75M [00:00<?, ?B/s]

added_tokens.json:   0%|          | 0.00/14.0 [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/104 [00:00<?, ?B/s]

--2024-01-09 20:53:33--  https://huggingface.co/HooshvareLab/gpt2-fa/resolve/main/pytorch_model.bin
Resolving huggingface.co (huggingface.co)... 18.239.50.16, 18.239.50.49, 18.239.50.103, ...
Connecting to huggingface.co (huggingface.co)|18.239.50.16|:443... connected.
HTTP request sent, awaiting response... 302 Found
Location: https://cdn-lfs.huggingface.co/HooshvareLab/gpt2-fa/46b0b806c740a0f0a9f056f5574c5fa896166fe844945fd3c849bf34365e5060?response-content-disposition=attachment%3B+filename*%3DUTF-8%27%27pytorch_model.bin%3B+filename%3D%22pytorch_model.bin%22%3B&response-content-type=application%2Foctet-stream&Expires=1705092813&Policy=eyJTdGF0ZW1lbnQiOlt7IkNvbmRpdGlvbiI6eyJEYXRlTGVzc1RoYW4iOnsiQVdTOkVwb2NoVGltZSI6MTcwNTA5MjgxM319LCJSZXNvdXJjZSI6Imh0dHBzOi8vY2RuLWxmcy5odWdnaW5nZmFjZS5jby9Ib29zaHZhcmVMYWIvZ3B0Mi1mYS80NmIwYjgwNmM3NDBhMGYwYTlmMDU2ZjU1NzRjNWZhODk2MTY2ZmU4NDQ5NDVmZDNjODQ5YmYzNDM2NWU1MDYwP3Jlc3BvbnNlLWNvbnRlbnQtZGlzcG9zaXRpb249KiZyZXNwb25zZS1jb250ZW50LXR5cGU9KiJ9XX0_&Sign

In [5]:
tokenizer = AutoTokenizer.from_pretrained(
    "/content/gpt2",
    bos_token='<s>',
    eos_token='</s>',
    pad_token='<pad>'
)

print(tokenizer.encode("<s>"))
print(tokenizer.encode("</s>"))
print(tokenizer.encode("<pad>"))
print(tokenizer.encode("<|startoftext|>"))
print(tokenizer.encode("<sep>"))


[0]
[2]
[1]
[6]
[9]


In [6]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [7]:
from torch.utils.data import Dataset  # this is the pytorch class import
import torch
torch.manual_seed(42)


class FerdosiPoemDataset(Dataset):

    def __init__(self, data_path, tokenizer, max_length=1024):

        f = open(data_path, 'r')
        self.tokenizer = tokenizer  # the gpt2 tokenizer we instantiated
        self.input_ids = []
        self.attn_masks = []

        lines = []
        while True:
            line = f.readline()
            if line == '':
                break # end of file
            lines.append(line)

        lines[0] = '<|startoftext|>' + lines[0]
        #print(len(lines))
        txt_list = [lines[i] + lines[i+1] for i in range(0, len(lines), 2)]
        print(txt_list[0:20])
        for i, txt in enumerate(txt_list):
            """
            This loop will iterate through each entry in the flavour text corpus.
            For each bit of text it will prepend it with the start of text token,
            then append the end of text token and pad to the maximum length with the
            pad token.
            """

            txt_list[i] = txt_list[i].replace("\n", "<sep>")
            encodings_dict = tokenizer('<s>' + txt + '</s>',
                                       truncation=True,
                                       max_length=max_length,
                                       padding="max_length")

            """
            Each iteration then appends either the encoded tensor to a list,
            or the attention mask for that encoding to a list. The attention mask is
            a binary list of 1's or 0's which determine whether the langauge model
            should take that token into consideration or not.
            """
            self.input_ids.append(torch.tensor(encodings_dict['input_ids']))
            self.attn_masks.append(torch.tensor(encodings_dict['attention_mask']))

    def __len__(self):
        return len(self.input_ids)

    def __getitem__(self, idx):
        if idx == self.__len__() - 1:
            labels = self.input_ids[idx]
        else:
            labels = self.input_ids[idx+1]
        return self.input_ids[idx], self.attn_masks[idx]

In [8]:
from torch.utils.data import random_split

max_seq = 25
data_path = '/content/drive/MyDrive/ferdousi.txt'
dataset = FerdosiPoemDataset(data_path, tokenizer, max_length=max_seq)

# Split into training and validation sets
train_size = int(0.9 * len(dataset))
val_size = len(dataset) - train_size
train_dataset, val_dataset = random_split(dataset, [train_size, val_size])

f'There are {len(train_dataset)} samples for training, and {len(val_dataset)} samples for validation testing'

['<|startoftext|>به نام خداوند جان و خرد\nکزین برتر اندیشه برنگذرد\n', 'خداوند نام و خداوند جای\nخداوند روزی ده رهنمای\n', 'خداوند کیوان و گردان سپهر\nفروزنده ماه و ناهید و مهر\n', 'ز نام و نشان و گمان برترست\nنگارندهٔ بر شده پیکرست\n', 'به بینندگان آفریننده را\nنبینی مرنجان دو بیننده را\n', 'نیابد بدو نیز اندیشه راه\nکه او برتر از نام و از جایگاه\n', 'سخن هر چه زین گوهران بگذرد\nنیابد بدو راه جان و خرد\n', 'خرد گر سخن برگزیند همی\nهمان را گزیند که بیند همی\n', 'ستودن نداند کس او را چو هست\nمیان بندگی را ببایدت بست\n', 'خرد را و جان را همی سنجد اوی\nدر اندیشهٔ سخته کی گنجد اوی\n', 'بدین آلت رای و جان و زبان\nستود آفریننده را کی توان\n', 'به هستیش باید که خستو شوی\nز گفتار بی کار یکسو شوی\n', 'پرستنده باشی و جوینده راه\nبه ژرفی به فرمانش کردن نگاه\n', 'توانا بود هر که دانا بود\nز دانش دل پیر برنا بود\n', 'از این پرده برتر سخن گاه نیست\nز هستی مر اندیشه را راه نیست\n', 'کنون ای خردمند وصف خرد\nبدین جایگه گفتن اندرخورد\n', 'کنون تا چه داری بیار از خرد\nکه گوش نیوشنده زو برخورد\n', 'خرد به

'There are 44647 samples for training, and 4961 samples for validation testing'

In [9]:
print(train_dataset[0])
print('----------------------------')
print(tokenizer.decode(train_dataset[0][0]))
print(tokenizer.decode(train_dataset[0][1]))

(tensor([    0, 10048,   274,   747, 10034,   326,  9908,   209,   800,   312,
        15520,   293,  6484,  1102, 22640,   907,   209,     2,     1,     1,
            1,     1,     1,     1,     1]), tensor([1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 0, 0, 0, 0, 0,
        0]))
----------------------------
<s>نمودم همه پیشت این جادویی
نه از تنبل و مکر وز بدخویی
</s><pad><pad><pad><pad><pad><pad><pad>
<pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><s><s><s><s><s><s><s>


In [10]:
from torch.utils.data import DataLoader, RandomSampler, SequentialSampler

train_dataloader = DataLoader(
    train_dataset,
    sampler=RandomSampler(train_dataset),
    batch_size=8
)

validation_dataloader = DataLoader(
    val_dataset,
    sampler=SequentialSampler(val_dataset),
    batch_size=8
)

In [11]:
import random
from transformers import GPT2LMHeadModel, GPT2Config
import numpy as np

# Loading the model configuration and setting it to the GPT2 standard settings.
configuration = GPT2Config.from_pretrained('./gpt2', output_hidden_states=False)

# Create the instance of the model and set the token size embedding length
model = GPT2LMHeadModel.from_pretrained("./gpt2", config=configuration)
model.resize_token_embeddings(len(tokenizer))

device = 'cuda' if torch.cuda.is_available() else 'cpu'
model.to(device)

# This step is optional but will enable reproducible runs.
seed_val = 42

random.seed(seed_val)
np.random.seed(seed_val)
torch.manual_seed(seed_val)
torch.cuda.manual_seed_all(seed_val)

In [12]:
epochs = 3
warmup_steps = 1e2
sample_every = 300

In [13]:
from transformers import AdamW

# AdamW is a class from the huggingface library, it is the optimizer we will be using, and we will only be instantiating it with the default parameters.
optimizer = AdamW(
    model.parameters(),
    lr=1e-4,
    eps=1e-8
)

/usr/local/lib/python3.10/dist-packages/transformers/optimization.py:429: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  warnings.warn(


In [14]:
from transformers import get_linear_schedule_with_warmup

"""
Total training steps is the number of data points, times the number of epochs.
Essentially, epochs are training cycles, how many times each point will be seen by the model.
"""

total_steps = len(train_dataloader) * epochs

"""
We can set a variable learning rate which will help scan larger areas of the
problem space at higher LR earlier, then fine tune to find the exact model minima
at lower LR later in training.
"""
scheduler = get_linear_schedule_with_warmup(
    optimizer,
    num_warmup_steps=warmup_steps,
    num_training_steps=total_steps)


In [15]:
sample_input = "<|startoftext|>"
sample_input_ids = torch.tensor(tokenizer([sample_input])["input_ids"])
sample_input_ids = sample_input_ids.to(device)

sample_outputs = model.generate(
    input_ids=sample_input_ids,
    # bos_token_id=random.randint(1, len(tokenizer.get_vocab())),
    do_sample=True,
    top_k=50,
    max_length=50,
    top_p=0.95,
    num_return_sequences=1
)
for i, sample_output in enumerate(sample_outputs):
    gen_sample_output = tokenizer.decode(sample_output, skip_special_tokens=False)
    gen_sample_output = gen_sample_output.replace("<|startoftext|>", "\n")
    gen_sample_output = gen_sample_output.replace("<s>", "")
    gen_sample_output = gen_sample_output.replace("</s>", "")
    gen_sample_output = gen_sample_output.replace("<sep>", "\n")

    print(f'Example output: {gen_sample_output}')

Example output: 
دها برای آموزش دادن و استفاده از استعدادشان برای توسعه دادن آن کسب و کار جدید هستند. «شاید برای برخی به فکر خرید یک گوشی موبایل در ابعاد بزرگ و یا حتی کوچک باشند. و یا اینکه بخواهند گوشی موبایل را با کمترین امکانات در


**در آموزش شبکه لیبل ها همان ورودی هستند چراکه نتیجه بسیار بهتری نسبت به حالتی که لیبل ها بیت بعدی باشند می دهند.**

In [16]:
import random
import time
import datetime
from tqdm import tqdm

training_stats = []

model = model.to(device)

for epoch_i in tqdm(range(0, epochs), position=0):

    print(f'Beginning epoch {epoch_i + 1} of {epochs}')

    total_train_loss = 0

    model.train()

    for step, batch in tqdm(enumerate(train_dataloader), total=len(train_dataloader), position=0):

        b_input_ids = batch[0].to(device)
        b_labels = batch[0].to(device)
        b_masks = batch[1].to(device)

        model.zero_grad()

        outputs = model(b_input_ids, labels=b_labels, attention_mask=b_masks, token_type_ids=None)

        # the first output of CausalLMOutputWithCrossAttentions object is the loss.
        loss = outputs[0]

        batch_loss = loss.item()
        total_train_loss += batch_loss

        loss.backward()
        optimizer.step()
        scheduler.step()

    # Calculate the average loss over all of the batches.
    avg_train_loss = total_train_loss / len(train_dataloader)
    print()
    print(f'Average Training Loss: {avg_train_loss}.')
    print()


    model.eval()

    nlls = []
    total_eval_loss = 0

    # Evaluate data for one epoch
    for batch in tqdm(validation_dataloader, total=len(validation_dataloader), position=0):

        b_input_ids = batch[0].to(device)
        b_labels = batch[0].to(device)
        b_masks = batch[1].to(device)

        with torch.no_grad():

            outputs = model(b_input_ids, attention_mask=b_masks, labels=b_labels)

            loss = outputs[0]

        batch_loss = loss.item()
        total_eval_loss += batch_loss

    avg_val_loss = total_eval_loss / len(validation_dataloader)
    nlls.append(loss)
    ppl = torch.exp(torch.stack(nlls).mean())

    print()
    print(f'Validation loss: {avg_val_loss}.')
    print(f'Perplexity : {ppl}')
    print('-------------------------------------')
    # Record all statistics from this epoch.
    training_stats.append(
        {
            'epoch': epoch_i + 1,
            'Training Loss': avg_train_loss,
            'Valid. Loss': avg_val_loss,
            'Perplexity Valid' : ppl
        }
    )


  0%|          | 0/3 [00:00<?, ?it/s]

Beginning epoch 1 of 3


100%|██████████| 5581/5581 [08:47<00:00, 10.59it/s]



Average Training Loss: 2.826789070726445.



 33%|███▎      | 1/3 [09:00<18:01, 540.73s/it]

...


Validation loss: 2.2430054767120287.
Perplexity : 7.430407524108887
-------------------------------------
Beginning epoch 3 of 3


100%|██████████| 5581/5581 [08:51<00:00, 10.50it/s]



Average Training Loss: 1.8828336963985068.



100%|██████████| 3/3 [27:10<00:00, 543.63s/it]


Validation loss: 2.2254637900182015.
Perplexity : 7.154715061187744
-------------------------------------


In [20]:
import re


def generator(text, max_length=128, num_return_sequences=3):
    model.eval()

    text = text.replace('\n', '<sep>')
    sample_input = "<|startoftext|>" + text
    print(text)
    sample_input_ids = torch.tensor(tokenizer([sample_input])["input_ids"])
    sample_input_ids = sample_input_ids.to(device)

    sample_outputs = model.generate(
        input_ids=sample_input_ids,
        # bos_token_id=random.randint(1, len(tokenizer.get_vocab())),
        do_sample=True,
        top_k=50,
        max_length=50,
        top_p=0.95,
        num_return_sequences=1
    )
    for i, sample_output in enumerate(sample_outputs):
        gen_sample_output = tokenizer.decode(sample_output, skip_special_tokens=False)
        gen_sample_output = gen_sample_output.replace("<|startoftext|>", "\n")
        gen_sample_output = gen_sample_output.replace("<s>", "")
        gen_sample_output = gen_sample_output.replace("</s>", "")
        gen_sample_output = gen_sample_output.replace("<sep>", "\n")

        print('example output:')
        print(gen_sample_output)

In [35]:
generator("بدو گفت بیژن که ای بدنژاد")
print('-----------------------')
generator("ز گهواره تا گور دانش بجوی")
print('-----------------------')
generator("کمند")
print('-----------------------')
generator('چنین گفت رستم به اسفندیار')

بدو گفت بیژن که ای بدنژاد
example output:

بدو گفت بیژن که ای بدنژاد
کجا باشدت از ما به نیکی نژاد

-----------------------
ز گهواره تا گور دانش بجوی
example output:

ز گهواره تا گور دانش بجوی
چه گوییم و این را چه پاسخ بگوی

-----------------------
کمند
example output:

کمندافگن بر نیزهٔ سرفراز
به روی هوا تیغ زد بر دراز

-----------------------
چنین گفت رستم به اسفندیار
example output:

چنین گفت رستم به اسفندیار
کز ایدر نیاید شما را سوار

